# Volatility Analysis with Cryptocurrency OHLC Data

## Understanding Volatility in Cryptocurrencies

Volatility is a key characteristic of cryptocurrency markets. It measures the degree of price fluctuations over time. Analyzing volatility is crucial for traders, investors, and risk managers.

## Steps to Perform Volatility Analysis

### Data Preparation

- Obtain OHLC Data: Collect historical OHLC (Open, High, Low, Close) data for the cryptocurrency you're analyzing.
- Data Cleaning: Ensure the data is clean, consistent, and free from errors or missing values.

### Calculate Returns

- Choose a Return Calculation Method
  - Log Returns: Often preferred for financial data due to their mathematical properties.
  - Simple Returns: Can be used but might introduce biases.
- Calculate Returns: Based on your chosen method, calculate **daily**, **weekly**, or **monthly** returns using the closing prices.

### Calculate Volatility Metrics

- Standard Deviation: The most common measure of volatility. Calculate the standard deviation of the returns over a specific period.
- Historical Volatility: An estimate of future volatility based on past price movements.
- Implied Volatility: Derived from option prices, it reflects market expectations of future volatility.
- Volatility Indexes: Use established volatility indexes like the VIX for comparison.

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import os
import glob  # Import glob to collect file names and use a list to store DataFrames.
import plotly.graph_objects as go

# Loading the CSV files from the specified directory
files = glob.glob("data/*.csv")

dfs = {}

for file in files:
    # Extract the file name (without extension) to use as the DataFrame key
    key = os.path.basename(file).split(".")[0].replace('-USD', '')

    df = pd.read_csv(
        file,
        parse_dates=['Date'],
        index_col='Date'
    )
    
    df.insert(0, 'Symbol', key)
    
    # Drop unnecessary column
    df.drop(columns=['Adj Close'],inplace=True)
    
    # Store each crypt
    dfs[key] = df

# Calculate daily returns for each cryptocurrency
for df in dfs.values():
    df['return'] = df['Close'].pct_change()

# Calculate Volatility and Sharpe ratio for each DataFrame
volatility = {key: df['return'].std() for key, df in dfs.items()}
annual_volatility = {key: volatility[key] * np.sqrt(252) for key in volatility}
sharp_ratio = {key: (np.mean(df['return']) * 252 - 0.03) / annual_volatility[key] for key, df in dfs.items()}

# Create a DataFrame from calculated values
data = {
    'Name': list(dfs.keys()), 
    'Volatility': list(volatility.values()), 
    'Annual Volatility': list(annual_volatility.values()), 
    'Sharp': list(sharp_ratio.values())
}
volatility_df = pd.DataFrame(data)

# Convert seaborn palette to hex
palette = sns.color_palette('Paired', len(volatility_df))
hex_palette = [f"rgb({int(r*255)},{int(g*255)},{int(b*255)})" for r, g, b in palette]

# Plotly bar plot for Annual Volatility
fig_volatility = go.Figure(
    data=[
        go.Bar(
            x=volatility_df['Name'], 
            y=volatility_df['Annual Volatility'], 
            marker=dict(color=hex_palette)
        )
    ]
)
fig_volatility.update_layout(
    title='Volatility for each coin',
    xaxis_title='Coin',
    yaxis_title='Annual Volatility',
    template='plotly_dark'
)
for i, v in enumerate(volatility_df['Annual Volatility']):
    fig_volatility.add_annotation(
        x=volatility_df['Name'][i], 
        y=v, 
        text=f"{v:.2f}", 
        showarrow=False, 
        font=dict(color='white')
    )
fig_volatility.show()

# Plotly bar plot for Sharpe ratio
fig_sharpe = go.Figure(
    data=[
        go.Bar(
            x=volatility_df['Name'], 
            y=volatility_df['Sharp'], 
            marker=dict(color=hex_palette)
        )
    ]
)
fig_sharpe.update_layout(
    title='Sharp ratio for each coin',
    xaxis_title='Coin',
    yaxis_title='Sharp Ratio',
    template='plotly_dark'
)
for i, v in enumerate(volatility_df['Sharp']):
    fig_sharpe.add_annotation(
        x=volatility_df['Name'][i], 
        y=v, 
        text=f"{v:.2f}", 
        showarrow=False, 
        font=dict(color='white')
    )
fig_sharpe.show()

### Advanced Volatility Analysis

- GARCH Models: Use Generalized Autoregressive Conditional Heteroskedasticity models to capture time-varying volatility.
- Risk Metrics: Employ Value at Risk (VaR) and Conditional Value at Risk (CVaR) to assess potential losses.

### Additional Considrations

- Time Period: The choice of time period (daily, weekly, monthly) affects volatility calculations.
- Data Frequency: Higher frequency data (e.g., minute-by-minute) can capture more volatility but requires careful handling.
- Volatility Clustering: Volatility tends to cluster, meaning periods of high volatility are often followed by other periods of high volatility.
- Leverage: Leverage amplifies volatility, so consider its impact when analyzing cryptocurrencies.